In [1]:
import pandas as pd
import numpy as np
import os 
import time

from sklearn.model_selection import train_test_split


from multiprocessing import Pool

from sklearn.decomposition import PCA 
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# # 读取数据
# data_all = pd.read_csv("data_all.csv")
# data_extra = pd.read_csv("data_extra.csv")

# # 删去无用的第一列
# data_all = data_all.iloc[:,1:]
# data_extra = data_extra.iloc[:,1:]

# # 删去植物园数据
# data_all = data_all.drop(columns=['植物园'])
# data_extra = data_extra.drop(columns=['植物园'])

# # 删去['PM2.5_24h','PM10_24h','SO2_24h','NO2_24h', 'O3_24h','CO_24h','PM10']
# data_all = data_all[~(data_all['type']=='PM10_24h')]
# data_all = data_all[~(data_all['type']=='PM2.5_24h')]
# data_all = data_all[~(data_all['type']=='PM10')]
# data_extra = data_extra[~(data_extra['type']=='SO2_24h')]
# data_extra = data_extra[~(data_extra['type']=='NO2_24h')]
# data_extra = data_extra[~(data_extra['type']=='CO_24h')]
# data_extra = data_extra[~(data_extra['type']=='O3_24h')]

In [3]:
# data_mix = pd.DataFrame()
# for date_now in np.unique(data_all['date'].values): 
#     df_date_all = data_all[data_all['date']==date_now]
#     df_date_extra = data_extra[data_extra['date']==date_now]
#     if not df_date_extra.shape[0]:
#         continue
        
#     for hour_now in np.unique(df_date_all['hour'].values):
#         df_date_hour_all = df_date_all[df_date_all['hour']==hour_now]
#         df_date_hour_extra = df_date_extra[df_date_extra['hour']==hour_now]
#         if df_date_hour_extra.shape[0]==4 and df_date_hour_all.shape[0]==2:       
#             df_mix_now = pd.concat([df_date_hour_all, df_date_hour_extra], axis=0)
#             data_mix = pd.concat([data_mix, df_mix_now], axis=0)
#             print(str(date_now)+' '+str(hour_now))
# data_mix.to_csv("data_mix.csv")

data_mix = pd.read_csv("data_mix.csv")
data_mix = data_mix.iloc[:,1:]

In [4]:
def data_pass(args):
    return data_cleaning(*args)

def data_cleaning(data_mix, loc):
    print(loc)
    
    data_mix['flag'] = True
    for type_now in np.unique(data_mix['type'].values):
        list_type_mix = data_mix[data_mix['type']==type_now].index.tolist()
        if np.isnan(data_mix.iloc[list_type_mix[0],loc]):
            data_mix['flag'].iloc[list_type_mix[0]] = False
        if np.isnan(data_mix.iloc[list_type_mix[-1],loc]):
            data_mix['flag'].iloc[list_type_mix[-1]] = False
        for idx in range(1,len(list_type_mix)-1): 
            if np.isnan(data_mix.iloc[list_type_mix[idx],loc]):
                mid = 0.5*(data_mix.iloc[list_type_mix[idx-1],loc]+data_mix.iloc[list_type_mix[idx+1],loc])
                if np.isnan(mid):
                    data_mix['flag'].iloc[list_type_mix[idx]] = False
                else:
                    data_mix.iloc[list_type_mix[idx],loc] = mid
                    
    return data_mix

In [5]:
MultiNum = 17
pool = Pool(processes=MultiNum)

num_loc = data_mix.shape[1]-3
para_data_mix = [data_mix]*num_loc
para_loc = list(range(3,data_mix.shape[1]))
para_list = list(zip(para_data_mix,para_loc))

start = time.time()
datamix_flag_list = pool.map(data_pass,para_list)
end = time.time()
print('training lasted: '+str(end-start))

pool.close() 
pool.join()

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
training lasted: 507.02945017814636


In [6]:
flag_result = pd.DataFrame()
data_mix_filled = data_mix.copy()
for loc in para_loc:
    data_mix_filled.iloc[:,loc] = datamix_flag_list[loc-3].iloc[:,loc]
    flag_result[str(loc)] = datamix_flag_list[loc-3]['flag']
flag_result['flag_res'] = False
for idx in range(flag_result.shape[0]):
    if flag_result.iloc[idx,:].sum() == num_loc:
        flag_result['flag_res'].iloc[idx] = True
data_mix_filled['flag'] = flag_result['flag_res']

In [7]:
for date_now in np.unique(data_mix_filled['date'].values): 
    list_date_mix = data_mix_filled[data_mix_filled['date']==date_now].index.tolist()
    for hour_now in np.unique(data_mix_filled.iloc[list_date_mix,1].values):
        list_hour_mix = data_mix_filled.iloc[list_date_mix,1][data_mix_filled.iloc[list_date_mix,1]==hour_now].index.tolist()
        if data_mix_filled['flag'].iloc[list_hour_mix].sum() < 6:
            data_mix_filled['flag'].iloc[list_hour_mix] = False

dataset_now = data_mix_filled[data_mix_filled['flag']]    
dataset_now = dataset_now.drop(['flag'],axis=1)

In [8]:
filename = 'data_mix_clean_all.csv'
dataset_now.to_csv(filename)

In [9]:
dataset_now

,date,hour,type,东四,天坛,官园,万寿西宫,奥体中心,农展馆,万柳,...,密云水库,东高村,永乐店,榆垡,琉璃河,前门,永定门内,西直门北,南三环,东四环
78,20140402,13,PM2.5,91.0,87.0,64.0,87.0,67.0,90.0,52.0,...,51.0,54.0,126.0,189.0,70.0,83.0,90.0,63.0,104.0,85.0
79,20140402,13,AQI,131.0,126.0,127.0,125.0,123.0,135.0,127.0,...,81.0,138.0,159.0,142.0,155.0,133.0,135.0,129.0,144.0,129.0
80,20140402,13,SO2,21.0,23.0,14.0,23.0,19.0,21.0,14.0,...,13.0,21.0,47.0,42.0,16.0,53.0,23.0,22.0,29.0,12.0
81,20140402,13,NO2,23.0,28.0,30.0,14.0,24.0,23.0,58.0,...,6.0,30.0,30.0,29.0,41.0,68.0,54.0,32.0,135.5,80.0
82,20140402,13,O3,153.0,152.0,150.0,169.0,147.0,162.0,109.0,...,147.0,128.0,150.0,148.0,126.0,45.0,120.0,120.0,60.0,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311671,20200509,23,AQI,66.0,60.0,63.0,64.0,63.0,63.0,64.0,...,37.0,66.0,74.0,57.0,61.0,54.0,46.0,73.0,60.0,65.0
311672,20200509,23,SO2,3.0,1.0,3.0,1.0,3.0,3.0,3.0,...,2.0,1.0,2.0,1.0,2.0,4.0,9.0,3.0,1.0,2.0
311673,20200509,23,NO2,57.0,23.0,41.0,47.0,47.0,46.0,49.0,...,25.0,61.0,49.0,29.0,34.0,53.0,49.0,48.0,58.0,50.0
311674,20200509,23,O3,2.0,1.0,8.0,2.0,8.0,3.0,2.0,...,41.0,1.0,1.0,19.0,6.0,2.0,1.0,5.0,2.0,1.0


In [ ]:
pca=PCA(n_components=1)
data_pca = dataset_now[dataset_now['type']=='PM2.5'].iloc[:,0:3]
data_pca.iloc[:,2] = 'PCA'
typelist = np.unique(dataset_now['type'].values).tolist()
feature_ori = np.zeros([dataset_now[dataset_now['type']=='PM2.5'].shape[0], len(typelist)])
for idx in range(len(typelist)):
    feature_ori[:,idx] = dataset_now[dataset_now['type']==typelist[idx]].iloc[:,3].values
feature_pca = pca.fit_transform(feature_ori)
data_pca[dataset_now.columns.values[-1]]=feature_pca

data_pca_plus = pd.DataFrame()
for idx in range(data_pca.shape[0]):
    data_pca_plus = pd.concat([data_pca_plus, data_pca.iloc[idx:idx+1,:]], axis=0)
    data_pca_plus = pd.concat([data_pca_plus, dataset_now[dataset_now['type']=='PM2.5'].iloc[idx:idx+1,:]], axis=0)
    

In [15]:
data_all

,date,hour,type,东四,天坛,官园,万寿西宫,奥体中心,农展馆,万柳,...,密云水库,东高村,永乐店,榆垡,琉璃河,前门,永定门内,西直门北,南三环,东四环
0,20140101,0,PM2.5,35.0,32.0,45.0,66.0,20.0,31.0,57.0,...,NaN,21.0,110.0,90.0,136.0,75.0,66.0,35.0,84.0,45.0
4,20140101,0,AQI,91.0,87.0,99.0,101.0,136.0,102.0,112.0,...,61.0,89.0,146.0,190.0,197.0,99.0,100.0,91.0,92.0,104.0
5,20140101,1,PM2.5,66.0,56.0,57.0,72.0,43.0,58.0,68.0,...,14.0,14.0,116.0,122.0,181.0,87.0,68.0,56.0,85.0,46.0
9,20140101,1,AQI,92.0,87.0,99.0,100.0,140.0,102.0,112.0,...,61.0,88.0,143.0,178.0,190.0,96.0,101.0,91.0,90.0,106.0
10,20140101,2,PM2.5,70.0,53.0,57.0,80.0,79.0,68.0,81.0,...,21.0,23.0,133.0,135.0,235.0,92.0,76.0,59.0,93.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270602,20200509,21,AQI,59.0,58.0,60.0,64.0,61.0,55.0,56.0,...,35.0,52.0,61.0,56.0,61.0,52.0,53.0,68.0,61.0,56.0
270603,20200509,22,PM2.5,45.0,41.0,39.0,47.0,45.0,46.0,42.0,...,22.0,42.0,49.0,30.0,38.0,28.0,34.0,54.0,43.0,44.0
270607,20200509,22,AQI,63.0,58.0,55.0,65.0,63.0,64.0,59.0,...,32.0,59.0,68.0,56.0,54.0,53.0,49.0,74.0,60.0,63.0
270608,20200509,23,PM2.5,48.0,43.0,45.0,46.0,45.0,45.0,46.0,...,26.0,48.0,54.0,33.0,44.0,38.0,32.0,53.0,43.0,47.0


In [16]:
data_extra

,date,hour,type,东四,天坛,官园,万寿西宫,奥体中心,农展馆,万柳,...,密云水库,东高村,永乐店,榆垡,琉璃河,前门,永定门内,西直门北,南三环,东四环
0,20140402,0,SO2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20140402,0,NO2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20140402,0,O3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20140402,0,CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20140402,1,SO2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415414,20200509,23,SO2,3.0,1.0,3.0,1.0,3.0,3.0,3.0,...,2.0,1.0,2.0,1.0,2.0,4.0,9.0,3.0,1.0,2.0
415416,20200509,23,NO2,57.0,23.0,41.0,47.0,47.0,46.0,49.0,...,25.0,61.0,49.0,29.0,34.0,53.0,49.0,48.0,58.0,50.0
415418,20200509,23,O3,2.0,1.0,8.0,2.0,8.0,3.0,2.0,...,41.0,1.0,1.0,19.0,6.0,2.0,1.0,5.0,2.0,1.0
415419,20200509,23,O3_24h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
